In [1]:
import numpy as np
import pandas as pd
from dobbel import dobbellogger

In [49]:
dob = dobbellogger()
dob.log(10, 200, 8, 125)
dob.download()
data = dob.datadf

Logging data for 10s
Done! The data is located in self.datadf


In [50]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2]  * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1/2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])

In [51]:
def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

In [52]:
def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

In [53]:
# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion, quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

In [54]:
def quat_to_euler(q):
    w, x, y, z = q[0], q[1], q[2], q[3]
    psi = np.arctan((2*x*y-2*w*z)/((2*w**2)+(2*x**2)-1))
    theta = -np.arcsin(2*x*z+2*w*y)
    phi = np.arctan((2*y*z-2*w*x)/((2*w**2)+(2*z**2)-1))
    return np.array([psi, theta, phi])

In [55]:
def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [57]:
def left_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, -q3, -q2],
        [q2, q3, 1, -q1],
        [q3, -q2, q1, 1]
    ])
    return lqm

def right_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, q3, q2],
        [q2, -q3, 1, q1],
        [q3, q2, -q1, 1]
    ])
    return lqm

In [58]:
y11 = np.array([data['x_acc'][0], data['y_acc'][0], data['z_acc'][0]])
gb = y11 / np.linalg.norm(y11)
gn = np.array([0, 0, 1])
A = - np.matmul(left_quat_mul(np.array([0, *gn])), right_quat_mul(np.array([0, *gb])))



# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(A)

# Eigenvalues are sorted in ascending order by np.linalg.eigh, 
# and the corresponding eigenvectors are columns of 'eigenvectors'
max_eigenvalue_index = np.argmax(eigenvalues)
max_eigenvector = eigenvectors[:, max_eigenvalue_index]

print("Matrix A:")
print(A)
print("Eigenvalues:")
print(eigenvalues)
print("Eigenvectors:")
print(eigenvectors)
print("Eigenvector corresponding to the largest eigenvalue:")
print(max_eigenvector)


Matrix A:
[[ 0.00239157 -0.9999168   0.01267534  1.        ]
 [-0.98724147 -1.00239157  0.99760843  0.98724147]
 [ 1.01259214 -0.99760843 -1.00239157  1.01259214]
 [-0.00239157  0.98724147 -1.01259214 -0.99760843]]
Eigenvalues:
[-2.34308813 -1.99903849 -0.47094108  1.8130677 ]
Eigenvectors:
[[-0.3829512   0.01784681 -0.7248257   0.57241381]
 [-0.41061609  0.73288628 -0.17145293 -0.51466107]
 [ 0.48827636  0.67917886  0.17013284  0.52091947]
 [ 0.66807859 -0.0357108  -0.64520262 -0.36892997]]
Eigenvector corresponding to the largest eigenvalue:
[ 0.57241381 -0.51466107  0.52091947 -0.36892997]


In [59]:
qtmin1tmin1 = max_eigenvector

dqde = np.array([
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

sigma_etha = np.array([
    [np.pi / 9, 0, 0],
    [0, np.pi / 9, 0],
    [0, 0, np.pi / 9]
])

Pttmin1 = 0.25 * np.matmul(np.matmul(np.matmul(np.matmul(right_quat_mul(qtmin1tmin1), dqde), sigma_etha), dqde.T), right_quat_mul(qtmin1tmin1))

In [60]:
gyro_bias = np.array([-0.145345332, 0.46132693, 0.01679937])
acc_bias = np.array([0.00682937, 0.0169035, -0.05057431])
gyro_noise = np.array([0, 0, 0])
acc_noise_p = np.array([0, 0, 0])
acc_noise_v = np.array([0, 0, 0])

v = np.array([0, 0, 0])
p = np.array([0, 0, 0])
q_list = [qtmin1tmin1]
p_list = [p]
v_list = [v]

# std van gyroscope
Q = np.array([
    [-0.14534332, 0, 0],
    [0, 0.46132693, 0],
    [0, 0, 0.01679937]
])

# std van accelerometer
R = np.array([
    [-0.00014147, 0, 0],
    [0, 0.00180437, 0],
    [0, 0, 0.00452501],
])

qtmin1tmin1 = np.reshape(qtmin1tmin1, 4)

for i in range(len(data['timestamp']) - 1):
    dt = (data['timestamp'][i + 1] - data['timestamp'][i]) / 1000
    gyro_measurement = np.array([data['x_gyro'][i], data['y_gyro'][i], data['z_gyro'][i]])
    omega = gyro_measurement - gyro_bias
    expq_omega = expq(omega, 0.5 * dt * np.pi / 180)
    
    Ftmin1 = right_quat_mul(expq_omega)
    Gtmin1 = - 0.5 * dt * np.matmul(left_quat_mul(qtmin1tmin1), dqde)
    
    Pttmin1 = np.matmul(np.matmul(Ftmin1, Pttmin1), Ftmin1.T) + np.matmul(np.matmul(Gtmin1, Q), Gtmin1.T)
    
    qttmin1 = quat_mul(qtmin1tmin1, expq_omega)
    G = - 0.5 * dt * np.matmul(left_quat_mul(qtmin1tmin1), dqde)
    
    q0, q1, q2, q3 = qttmin1
    # Zou kunnen dat hier de geconjugeerde genomen moet worden!!
    H = np.array([
        [2 * q2, 2 * q3, 2 * q0, 2 * q1],
        [-2 * q1, -2 * q0, 2 * q3, 2 * q2],
        [4 * q0, 0, 0, 4 * q3]
    ])
    S = np.matmul(np.matmul(H, Pttmin1), H.T) + R
    K = np.matmul(np.matmul(Pttmin1, H.T), np.linalg.inv(S))
    
    yat = np.array([data['x_acc'][i], data['y_acc'][i], data['z_acc'][i]]) - acc_bias
    yattmin1 = rotate_vector(gn, qttmin1) # Zou kunnen dat hier een min achter moet!!!
    epsilon = yat - yattmin1
    
    q_tt_measure = np.reshape(qttmin1 + np.matmul(K, epsilon), (4, 1))
    P_tt_measure = Pttmin1 - np.matmul(np.matmul(K, S), K.T)

    qtt = q_tt_measure / ((np.linalg.norm(q_tt_measure)))
    J = np.matmul(q_tt_measure, q_tt_measure.T) / (np.linalg.norm(q_tt_measure) ** 3)
    Ptt = np.matmul(np.matmul(J, P_tt_measure), J.T)
    
    Pttmin1 = Ptt
    qtmin1tmin1 = np.reshape(qtt, 4)
    
    q_list.append(qtt)
    

In [61]:
for q, timestamp in zip(q_list, data['timestamp']):
    print(timestamp, quat_to_euler(q) * 180 / np.pi)

0.0 [-31.59519896 -77.45106141 -70.51710817]
25.0 [[ 65.54581057]
 [-73.42271502]
 [ 23.15504405]]
31.0 [[ 58.8827599 ]
 [-72.7179236 ]
 [ 29.58197483]]
35.0 [[ 58.12958098]
 [-72.57862821]
 [ 30.30465928]]
41.0 [[ 58.47824572]
 [-72.49820479]
 [ 29.96077416]]
46.0 [[ 59.5099177 ]
 [-72.52971263]
 [ 28.9612932 ]]
50.0 [[ 60.47879454]
 [-72.57693315]
 [ 28.02375688]]
56.0 [[ 63.14735154]
 [-72.70360779]
 [ 25.439278  ]]
60.0 [[ 64.41191369]
 [-72.75226309]
 [ 24.2148099 ]]
66.0 [[ 67.51292161]
 [-72.84247227]
 [ 21.21278063]]
70.0 [[ 68.87235442]
 [-72.86214183]
 [ 19.89583341]]
76.0 [[ 72.01379155]
 [-72.87281801]
 [ 16.85235975]]
81.0 [[ 74.09476866]
 [-72.84419616]
 [ 14.83746106]]
85.0 [[ 75.39613346]
 [-72.81135659]
 [ 13.57762595]]
91.0 [[ 78.2441921 ]
 [-72.70263108]
 [ 10.81924288]]
95.0 [[ 79.43254075]
 [-72.63784229]
 [  9.66868478]]
101.0 [[ 82.03707273]
 [-72.46550313]
 [  7.15003078]]
106.0 [[ 83.6450654 ]
 [-72.3246383 ]
 [  5.59534406]]
112.0 [[ 85.7822403 ]
 [-72.0970412